# **Preprocessing**

Tahap ini bertujuan untuk menyiapkan data sebelum dianalisis. Data diambil dari dua sumber berbeda (MySQL dan PostgreSQL), lalu digabungkan menjadi satu dataframe. Hasil akhirnya adalah dataset Iris dengan lima kolom utama: sepal_length, sepal_width, petal_length, petal_width, dan species.

## **Outliers Cleaning**

Tahap ini fokus pada pembersihan data dengan mendeteksi nilai pencilan (outlier). Outlier dapat mengganggu analisis karena berbeda jauh dari distribusi data utama. Pembersihan dilakukan dengan metode Anomaly Detection agar data lebih bersih dan representatif.

In [ ]:
import pandas as pd
from module.dataTransformer import combineData
from module.fetcher import fetchDataMysql, fetchDataPg

data_pg = fetchDataPg("SELECT petal_length, petal_width, species FROM iris_table")
data_my = fetchDataMysql("SELECT sepal_length, sepal_width FROM iris_table")

iris_df = combineData(data1=data_my, data2=data_pg)
iris_df

|     | sepal_length | sepal_width | petal_length | petal_width | species        |
|-----|--------------|-------------|--------------|-------------|----------------|
| 0   | 5.10         | 3.50        | 1.40         | 0.20        | Iris-setosa    |
| 1   | 4.90         | 3.00        | 1.40         | 0.20        | Iris-setosa    |
| 2   | 4.70         | 3.20        | 1.30         | 0.20        | Iris-setosa    |
| 3   | 4.60         | 3.10        | 1.50         | 0.20        | Iris-setosa    |
| 4   | 5.00         | 3.60        | 1.40         | 0.20        | Iris-setosa    |
| ... | ...          | ...         | ...          | ...         | ...            |
| 145 | 6.70         | 3.00        | 5.20         | 2.30        | Iris-virginica |
| 146 | 6.30         | 2.50        | 5.00         | 1.90        | Iris-virginica |
| 147 | 6.50         | 3.00        | 5.20         | 2.00        | Iris-virginica |
| 148 | 6.20         | 3.40        | 5.40         | 2.30        | Iris-virginica |
| 149 | 5.90         | 3.00        | 5.10         | 1.80        | Iris-virginica |

150 rows × 5 columns


In [ ]:
# Menyingkirkan kolom species (class)
numeric_iris_df = iris_df[["sepal_length", "sepal_width", "petal_length", "petal_width"]].astype(float)
numeric_iris_df

|     | sepal_length | sepal_width | petal_length | petal_width |
|-----|--------------|-------------|--------------|-------------|
| 0   | 5.1          | 3.5         | 1.4          | 0.2         |
| 1   | 4.9          | 3.0         | 1.4          | 0.2         |
| 2   | 4.7          | 3.2         | 1.3          | 0.2         |
| 3   | 4.6          | 3.1         | 1.5          | 0.2         |
| 4   | 5.0          | 3.6         | 1.4          | 0.2         |
| ... | ...          | ...         | ...          | ...         |
| 145 | 6.7          | 3.0         | 5.2          | 2.3         |
| 146 | 6.3          | 2.5         | 5.0          | 1.9         |
| 147 | 6.5          | 3.0         | 5.2          | 2.0         |
| 148 | 6.2          | 3.4         | 5.4          | 2.3         |
| 149 | 5.9          | 3.0         | 5.1          | 1.8         |

150 rows × 4 columns


## **Model ABOD**

Metode Angle-Based Outlier Detection (ABOD) digunakan untuk mendeteksi outlier dengan mengukur variasi sudut antar titik data. Parameter fraction=0.05 artinya sekitar 5% data dianggap sebagai outlier. Model ini cocok untuk dataset berdimensi rendah seperti Iris.

In [ ]:
from pycaret.anomaly import *

s = setup(data=numeric_iris_df)

abod_model = create_model("abod", fraction=0.05)

df_abod = assign_model(abod_model)

df_abod

**Data IRIS clean**

Data Tanpa Outliers

Setelah menjalankan ABOD, data yang terdeteksi sebagai inlier (Anomaly = 0) dipertahankan. Hasilnya adalah dataset Iris tanpa outlier, sehingga lebih stabil untuk analisis selanjutnya.

In [ ]:
clean_iris_abod = df_abod[df_abod["Anomaly"] == 0].merge(iris_df["species"], left_index=True, right_index=True)[["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]]
clean_iris_abod

|     | sepal_length | sepal_width | petal_length | petal_width | species        |
|-----|--------------|-------------|--------------|-------------|----------------|
| 0   | 5.1          | 3.5         | 1.4          | 0.2         | Iris-setosa    |
| 1   | 4.9          | 3.0         | 1.4          | 0.2         | Iris-setosa    |
| 2   | 4.7          | 3.2         | 1.3          | 0.2         | Iris-setosa    |
| 3   | 4.6          | 3.1         | 1.5          | 0.2         | Iris-setosa    |
| 4   | 5.0          | 3.6         | 1.4          | 0.2         | Iris-setosa    |
| ... | ...          | ...         | ...          | ...         | ...            |
| 145 | 6.7          | 3.0         | 5.2          | 2.3         | Iris-virginica |
| 146 | 6.3          | 2.5         | 5.0          | 1.9         | Iris-virginica |
| 147 | 6.5          | 3.0         | 5.2          | 2.0         | Iris-virginica |
| 148 | 6.2          | 3.4         | 5.4          | 2.3         | Iris-virginica |
| 149 | 5.9          | 3.0         | 5.1          | 1.8         | Iris-virginica |

150 rows × 5 columns


## **Metode KNN**

Metode K-Nearest Neighbors (KNN) untuk anomaly detection digunakan untuk mengidentifikasi data yang berbeda jauh dari tetangga terdekatnya. Sama seperti ABOD, digunakan fraction=0.05 untuk menganggap 5% data sebagai outlier.

In [ ]:
from pycaret.anomaly import *

s = setup(data=numeric_iris_df)

iforest_model = create_model("knn", fraction=0.05)

df_knn = assign_model(iforest_model)

df_knn

<table style="border-collapse: collapse; width: 60%; text-align: left; font-family: Arial;">
  <tr style="background-color: #d9e1e8;">
    <th style="padding: 8px;">Description</th>
    <th style="padding: 8px;">Value</th>
  </tr>
  <tr style="background-color: #ffffff;">
    <td>Session id</td>
    <td>5315</td>
  </tr>
  <tr style="background-color: #d9e1e8;">
    <td>Original data shape</td>
    <td>(150, 4)</td>
  </tr>
  <tr style="background-color: #ffffff;">
    <td>Transformed data shape</td>
    <td>(150, 4)</td>
  </tr>
  <tr style="background-color: #d9e1e8;">
    <td>Numeric features</td>
    <td>4</td>
  </tr>
  <tr style="background-color: #90ee90;"> <!-- hijau -->
    <td>Preprocess</td>
    <td>True</td>
  </tr>
  <tr style="background-color: #ffffff;">
    <td>Imputation type</td>
    <td>simple</td>
  </tr>
  <tr style="background-color: #d9e1e8;">
    <td>Numeric imputation</td>
    <td>mean</td>
  </tr>
  <tr style="background-color: #ffffff;">
    <td>Categorical imputation</td>
    <td>mode</td>
  </tr>
  <tr style="background-color: #d9e1e8;">
    <td>CPU Jobs</td>
    <td>-1</td>
  </tr>
  <tr style="background-color: #ffffff;">
    <td>Use GPU</td>
    <td>False</td>
  </tr>
  <tr style="background-color: #d9e1e8;">
    <td>Log Experiment</td>
    <td>False</td>
  </tr>
  <tr style="background-color: #ffffff;">
    <td>Experiment Name</td>
    <td>anomaly-default-name</td>
  </tr>
  <tr style="background-color: #d9e1e8;">
    <td>USI</td>
    <td>7446</td>
  </tr>
</table>


| sepal_length | sepal_width | petal_length | petal_width | Anomaly | Anomaly_Score |
|--------------|-------------|--------------|-------------|---------|---------------|
| 5.1          | 3.5         | 1.4          | 0.2         | 0       | 0.141421      |
| 4.9          | 3.0         | 1.4          | 0.2         | 0       | 0.173205      |
| 4.7          | 3.2         | 1.3          | 0.2         | 0       | 0.264575      |
| 4.6          | 3.1         | 1.5          | 0.2         | 0       | 0.264575      |
| 5.0          | 3.6         | 1.4          | 0.2         | 0       | 0.244949      |
| ...          | ...         | ...          | ...         | ...     | ...           |
| 6.7          | 3.0         | 5.2          | 2.3         | 0       | 0.374166      |
| 6.3          | 2.5         | 5.0          | 1.9         | 0       | 0.479583      |
| 6.5          | 3.0         | 5.2          | 2.0         | 0       | 0.387298      |
| 6.2          | 3.4         | 5.4          | 2.3         | 0       | 0.624500      |
| 5.9          | 3.0         | 5.1          | 1.8         | 0       | 0.360555      |

150 rows × 6 columns



**Data IRIS Clean**

Data tanpa outliers

Data bersih dari metode KNN diperoleh dengan menyaring baris yang bukan outlier (Anomaly = 0). Hasil akhirnya adalah dataset Iris yang lebih konsisten, bebas dari nilai pencilan, dan siap dipakai untuk analisis atau model klasifikasi.

In [1]:
clean_iris_knn = df_knn[df_knn["Anomaly"] == 0].merge(iris_df["species"], left_index=True, right_index=True)[["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]]
clean_iris_knn

| sepal_length | sepal_width | petal_length | petal_width | species        |
|--------------|-------------|--------------|-------------|----------------|
| 5.1          | 3.5         | 1.4          | 0.2         | Iris-setosa    |
| 4.9          | 3.0         | 1.4          | 0.2         | Iris-setosa    |
| 4.7          | 3.2         | 1.3          | 0.2         | Iris-setosa    |
| 4.6          | 3.1         | 1.5          | 0.2         | Iris-setosa    |
| 5.0          | 3.6         | 1.4          | 0.2         | Iris-setosa    |
| ...          | ...         | ...          | ...         | ...            |
| 6.7          | 3.0         | 5.2          | 2.3         | Iris-virginica |
| 6.3          | 2.5         | 5.0          | 1.9         | Iris-virginica |
| 6.5          | 3.0         | 5.2          | 2.0         | Iris-virginica |
| 6.2          | 3.4         | 5.4          | 2.3         | Iris-virginica |
| 5.9          | 3.0         | 5.1          | 1.8         | Iris-virginica |

142 rows × 5 columns
